In [208]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [209]:
import json


filename = "/content/drive/MyDrive/compoundV2_transactions_ethereum_chunk_2 (1).json"

with open(filename, 'r') as f:
    chunk3 = json.load(f)



In [210]:
filename = "/content/drive/MyDrive/compoundV2_transactions_ethereum_chunk_1 (1).json"

with open(filename, 'r') as f:
    chunk1 = json.load(f)
filename = "/content/drive/MyDrive/compoundV2_transactions_ethereum_chunk_0 (1).json"

with open(filename, 'r') as f:
    chunk2 = json.load(f)



In [212]:
filename = "/content/drive/MyDrive/compoundV2_transactions_ethereum_chunk_3 (1).json"

with open(filename, 'r') as f:
    chunk4 = json.load(f)

filename = "/content/drive/MyDrive/compoundV2_transactions_ethereum_chunk_4 (1).json"

with open(filename, 'r') as f:
    chunk5 = json.load(f)





In [213]:
import pandas as pd


deposits_df = pd.json_normalize(chunk1['deposits'])
withdraws_df = pd.json_normalize(chunk1['withdraws'])
borrows_df = pd.json_normalize(chunk1['borrows'])
repays_df = pd.json_normalize(chunk1['repays'])

In [214]:
deposits_df1 = pd.json_normalize(chunk2['deposits'])
withdraws_df1 = pd.json_normalize(chunk2['withdraws'])
borrows_df1 = pd.json_normalize(chunk2['borrows'])
repays_df1 = pd.json_normalize(chunk2['repays'])

deposits_df2 = pd.json_normalize(chunk3['deposits'])
withdraws_df2 = pd.json_normalize(chunk3['withdraws'])
borrows_df2 = pd.json_normalize(chunk3['borrows'])
repays_df2 = pd.json_normalize(chunk3['repays'])

In [215]:
deposits_df4 = pd.json_normalize(chunk4['deposits'])
withdraws_df4 = pd.json_normalize(chunk4['withdraws'])
borrows_df4 = pd.json_normalize(chunk4['borrows'])
repays_df4 = pd.json_normalize(chunk4['repays'])
liquidates_df4 = pd.json_normalize(chunk4['liquidates'])


deposits_df5 = pd.json_normalize(chunk5['deposits'])
withdraws_df5 = pd.json_normalize(chunk5['withdraws'])
borrows_df5 = pd.json_normalize(chunk5['borrows'])
repays_df5 = pd.json_normalize(chunk5['repays'])
liquidates_df5 = pd.json_normalize(chunk5['liquidates'])




In [216]:
liquid_df1 = pd.json_normalize(chunk1['liquidates'])
liquid_df2 = pd.json_normalize(chunk2['liquidates'])
liquid_df3 = pd.json_normalize(chunk3['liquidates'])


In [217]:
deposits_df = pd.concat([deposits_df, deposits_df1, deposits_df2, deposits_df4, deposits_df5], ignore_index=True)


withdraws_df = pd.concat([withdraws_df, withdraws_df1, withdraws_df2, withdraws_df4, withdraws_df5], ignore_index=True)


borrows_df = pd.concat([borrows_df, borrows_df1, borrows_df2, borrows_df4, borrows_df5], ignore_index=True)


repays_df = pd.concat([repays_df, repays_df1, repays_df2, repays_df4, repays_df5], ignore_index=True)

liquidates_df = pd.concat([liquid_df1, liquid_df2, liquid_df3, liquidates_df4, liquidates_df5], ignore_index=True)

In [218]:

liquidates_df.drop(columns=["amount","hash","timestamp","asset.id","asset.symbol","liquidator.id"], inplace=True)


In [219]:

liquidates_df['amountUSD'] = pd.to_numeric(liquidates_df['amountUSD'], errors='coerce')
liquidates_df.dropna(subset=['amountUSD'], inplace=True)
liquidates_df['amountUSD'] = liquidates_df['amountUSD'].astype(int)

In [220]:
liquidates_df.drop(columns=["id"],inplace=True)

In [221]:
deposits_df.rename(columns={'account.id': 'wallet'}, inplace=True)
withdraws_df.rename(columns={'account.id': 'wallet'}, inplace=True)
borrows_df.rename(columns={'account.id': 'wallet'}, inplace=True)
repays_df.rename(columns={'account.id': 'wallet'}, inplace=True)


In [222]:

def summarize(df, name):
    return df.groupby('wallet').agg(
        **{
            f'n_{name}': ('wallet', 'count'),
            f'total_{name}_usd': ('amountUSD', 'sum')
        }
    )
deposit_summary = summarize(deposits_df, 'deposits')
withdraw_summary = summarize(withdraws_df, 'withdraws')
borrow_summary = summarize(borrows_df, 'borrows')
repay_summary = summarize(repays_df, 'repays')

# Merge all summaries into a single DataFrame
wallet_df = deposit_summary.join([withdraw_summary, borrow_summary, repay_summary], how='outer')
wallet_df.fillna(0, inplace=True)

wallet_df.reset_index(inplace=True)

In [223]:
print(deposits_df.columns)

Index(['amount', 'amountUSD', 'hash', 'id', 'timestamp', 'wallet', 'asset.id',
       'asset.symbol'],
      dtype='object')


In [224]:
liquidates_df.rename(columns={"liquidatee.id": "wallet"}, inplace=True)

#  Group by wallet and sum amountUSD for total negative balance
liquidation_summary = liquidates_df.groupby("wallet")["amountUSD"].sum().reset_index()
liquidation_summary.rename(columns={"amountUSD": "negative_balance"}, inplace=True)


wallet_df = wallet_df.merge(liquidation_summary, on="wallet", how="left")


wallet_df["negative_balance"] = wallet_df["negative_balance"].fillna(0)

In [225]:
for df in [deposits_df, withdraws_df, borrows_df, repays_df]:
    df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce')
    df.dropna(subset=['amountUSD'], inplace=True)
    df['amountUSD'] = df['amountUSD'].astype(int)

In [226]:
wallet_df

,wallet,n_deposits,total_deposits_usd,n_withdraws,total_withdraws_usd,n_borrows,total_borrows_usd,n_repays,total_repays_usd,negative_balance
0,0x000000aaee6a496aaf7b7452518781786313400f,110.0,13820.2292488350302145540529593969113692.27765...,177.0,5468.5520380918352776295314394564975395.835847...,157.0,25591.1029000000039537569562075.12500000000235...,138.0,9.230594761774815893143813.8088870201515018644...,412124.0
1,0x00000a6bbb35ea3f6f8dbc94801738f490c30a8d,2.0,1.0057405024263636945.058038216181813887183738...,2.0,1.0057413067689067381834629513248755.035790593...,0.0,0,0.0,0,0.0
2,0x000206732258d7511fa624127228e6a032718e62,1.0,256.457256015527162855472,1.0,258.2361211126273856122385595885689,0.0,0,0.0,0,0.0
3,0x0004d2a2f9a823c1a585fde6514a17ff695e0001,1.0,54598.76122820483647051154535875845,5.0,10140.559404388063487036.210031045027248931495...,0.0,0,0.0,0,0.0
4,0x0006e4548aed4502ec8c844567840ce6ef1013f5,30.0,6172.3306775991096355898060088585444107.331247...,62.0,100006.53373290539429931733355816653161612769....,94.0,200004994.64177813837936532787.059713173200162...,252.0,25785.046271702711191178279697367782202.404436...,0.0
...,...,...,...,...,...,...,...,...,...,...
23032,0xfdf81d551a6779bed6905d8e7d9fb6cefcfb219a,0.0,0,0.0,0,0.0,0,2.0,1636.3717624280,0.0
23033,0xfe6a6290186c300ae4252eeb61382e62322f48bc,0.0,0,0.0,0,0.0,0,2.0,404.114316767462151350523693.74,0.0
23034,0xfed7fe268c4062151a7179772095b80ab08cb0ee,0.0,0,0.0,0,0.0,0,10.0,728.070343194882961537237468275887874039931051...,0.0
23035,0xff205781433e439729801b2ca5a966f58747c254,0.0,0,0.0,0,0.0,0,3.0,1.1293338168900361671656850793374770.066772732...,0.0


In [227]:
wallet_df['total_deposits_usd'] = pd.to_numeric(wallet_df['total_deposits_usd'], errors='coerce')
wallet_df.dropna(subset=['total_deposits_usd'], inplace=True)
wallet_df['total_deposits_usd'] = wallet_df['total_deposits_usd'].astype(int)

wallet_df['total_withdraws_usd'] = pd.to_numeric(wallet_df['total_withdraws_usd'], errors='coerce')
wallet_df.dropna(subset=['total_withdraws_usd'], inplace=True)
wallet_df['total_withdraws_usd'] = wallet_df['total_withdraws_usd'].astype(int)

wallet_df['total_borrows_usd'] = pd.to_numeric(wallet_df['total_borrows_usd'], errors='coerce')
wallet_df.dropna(subset=['total_borrows_usd'], inplace=True)
wallet_df['total_borrows_usd'] = wallet_df['total_borrows_usd'].astype(int)

wallet_df['total_repays_usd'] = pd.to_numeric(wallet_df['total_repays_usd'], errors='coerce')
wallet_df.dropna(subset=['total_repays_usd'], inplace=True)
wallet_df['total_repays_usd'] = wallet_df['total_repays_usd'].astype(int)

In [228]:
wallet_df["repay_to_borrow_ratio"] = wallet_df["total_repays_usd"] / (wallet_df["total_borrows_usd"] + 1)
wallet_df["deposit_to_borrow_ratio"] = wallet_df["total_deposits_usd"] / (wallet_df["total_borrows_usd"] + 1)
wallet_df["liquidation_ratio"] = wallet_df["negative_balance"] / (wallet_df["total_borrows_usd"] + 1)
wallet_df["activity_count"] = (
    wallet_df["n_deposits"] +
    wallet_df["n_withdraws"] +
    wallet_df["n_borrows"] +
    wallet_df["n_repays"]
)



In [229]:
wallet_df

,wallet,n_deposits,total_deposits_usd,n_withdraws,total_withdraws_usd,n_borrows,total_borrows_usd,n_repays,total_repays_usd,negative_balance,repay_to_borrow_ratio,deposit_to_borrow_ratio,liquidation_ratio,activity_count
2,0x000206732258d7511fa624127228e6a032718e62,1.0,256,1.0,258,0.0,0,0.0,0,0.0,0.0,256.0,0.0,2.0
7,0x000ea36028ca0a4d56778c5333c3f164efd7e25c,1.0,1,1.0,1,0.0,0,0.0,0,0.0,0.0,1.0,0.0,2.0
9,0x00164af1287c58d244049de5a6605b930e2cdd28,1.0,112,1.0,112,0.0,0,0.0,0,0.0,0.0,112.0,0.0,2.0
13,0x002c83ebc32d18ff4f74852565402cf7bde35ccc,1.0,100,0.0,0,0.0,0,0.0,0,0.0,0.0,100.0,0.0,1.0
14,0x002df1fb52465e43cdcb98c3598c9aab4a17d9d8,1.0,2,0.0,0,0.0,0,0.0,0,0.0,0.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0xfdb474ff09b55e9714c5697d01da226301de5e52,0.0,0,0.0,0,0.0,0,1.0,300,0.0,300.0,0.0,0.0,1.0
23030,0xfddc5d0aa495af31c106b5b8bd11bac5020c0188,0.0,0,0.0,0,0.0,0,1.0,19444,0.0,19444.0,0.0,0.0,1.0
23031,0xfdec24a1d6361698c0f0d44e47d47da24ecc7eca,0.0,0,0.0,0,0.0,0,1.0,30,0.0,30.0,0.0,0.0,1.0
23032,0xfdf81d551a6779bed6905d8e7d9fb6cefcfb219a,0.0,0,0.0,0,0.0,0,2.0,1636,0.0,1636.0,0.0,0.0,2.0


In [230]:
from sklearn.preprocessing import MinMaxScaler
scoring_features = wallet_df[[
    "repay_to_borrow_ratio",
    "deposit_to_borrow_ratio",
    "total_repays_usd",
    "activity_count",
    "liquidation_ratio"
]]

scaler = MinMaxScaler()
scaled = scaler.fit_transform(scoring_features)

# Apply weights: higher is better for all except liquidation
weights = [0.3, 0.2, 0.2, 0.1, -0.3]  # Negative weight for liquidation
scores = (scaled * weights).sum(axis=1)


wallet_df["credit_score"] = MinMaxScaler().fit_transform(scores.reshape(-1, 1)) * 100

In [231]:
wallet_df

,wallet,n_deposits,total_deposits_usd,n_withdraws,total_withdraws_usd,n_borrows,total_borrows_usd,n_repays,total_repays_usd,negative_balance,repay_to_borrow_ratio,deposit_to_borrow_ratio,liquidation_ratio,activity_count,credit_score
2,0x000206732258d7511fa624127228e6a032718e62,1.0,256,1.0,258,0.0,0,0.0,0,0.0,0.0,256.0,0.0,2.0,88.860848
7,0x000ea36028ca0a4d56778c5333c3f164efd7e25c,1.0,1,1.0,1,0.0,0,0.0,0,0.0,0.0,1.0,0.0,2.0,88.860848
9,0x00164af1287c58d244049de5a6605b930e2cdd28,1.0,112,1.0,112,0.0,0,0.0,0,0.0,0.0,112.0,0.0,2.0,88.860848
13,0x002c83ebc32d18ff4f74852565402cf7bde35ccc,1.0,100,0.0,0,0.0,0,0.0,0,0.0,0.0,100.0,0.0,1.0,88.205603
14,0x002df1fb52465e43cdcb98c3598c9aab4a17d9d8,1.0,2,0.0,0,0.0,0,0.0,0,0.0,0.0,2.0,0.0,1.0,88.205603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0xfdb474ff09b55e9714c5697d01da226301de5e52,0.0,0,0.0,0,0.0,0,1.0,300,0.0,300.0,0.0,0.0,1.0,88.205603
23030,0xfddc5d0aa495af31c106b5b8bd11bac5020c0188,0.0,0,0.0,0,0.0,0,1.0,19444,0.0,19444.0,0.0,0.0,1.0,88.205603
23031,0xfdec24a1d6361698c0f0d44e47d47da24ecc7eca,0.0,0,0.0,0,0.0,0,1.0,30,0.0,30.0,0.0,0.0,1.0,88.205603
23032,0xfdf81d551a6779bed6905d8e7d9fb6cefcfb219a,0.0,0,0.0,0,0.0,0,2.0,1636,0.0,1636.0,0.0,0.0,2.0,88.860848


In [246]:
top_1000_wallets = wallet_df.sort_values(by="credit_score", ascending=False).head(1000)


In [240]:
top_1000_wallets=top_1000_wallets[["wallet","credit_score"]]


In [241]:
top_1000_wallets.to_csv("top_1000_wallet_scores1.csv", index=False)


In [245]:
top_1000_wallets

,wallet,n_deposits,total_deposits_usd,n_withdraws,total_withdraws_usd,n_borrows,total_borrows_usd,n_repays,total_repays_usd,negative_balance,repay_to_borrow_ratio,deposit_to_borrow_ratio,liquidation_ratio,activity_count,credit_score
22883,0xbf84f317bd659451dbffac43efac58f0066fd4e9,0.0,0,0.0,0,0.0,0,5.0,-9223372036854775808,0.0,-9.223372e+18,0.000000e+00,0.0,5.0,0.000000
22499,0x2fb5ba1766c44d9568ccb1dfc50e4426c8e3ce3d,0.0,0,0.0,0,0.0,0,1.0,14690,454018.0,1.469000e+04,0.000000e+00,454018.0,1.0,33.165086
14023,0x4c7551d0f2911aa3639cbab0232081dc71b02b01,0.0,0,1.0,50,0.0,0,0.0,0,341216.0,0.000000e+00,0.000000e+00,341216.0,1.0,46.840051
7539,0x97b401ae5db9f3159df1c639d15f719ef4a4a6dd,4.0,-9223372036854775808,1.0,300147,0.0,0,0.0,0,0.0,0.000000e+00,-9.223372e+18,0.0,5.0,54.140858
8203,0xa44038533f10bf668207c7916ad341400a75a70c,5.0,-9223372036854775808,0.0,0,0.0,0,0.0,0,0.0,0.000000e+00,-9.223372e+18,0.0,5.0,54.140858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15855,0xb96f8caecd37e663fb5f75f680fe2d6fa76a2fad,0.0,0,1.0,5,0.0,0,0.0,0,0.0,0.000000e+00,0.000000e+00,0.0,1.0,88.205603
7250,0x92231ff5a2262708101dd8d47c2e46339a788390,1.0,19,0.0,0,0.0,0,0.0,0,0.0,0.000000e+00,1.900000e+01,0.0,1.0,88.205603
7248,0x9221d277d6c74d07aa321aad9df75d432c925143,1.0,21,0.0,0,0.0,0,0.0,0,0.0,0.000000e+00,2.100000e+01,0.0,1.0,88.205603
7247,0x92218be18b431bbd3e9972042aa3a633c7d0ef56,1.0,5,0.0,0,0.0,0,0.0,0,0.0,0.000000e+00,5.000000e+00,0.0,1.0,88.205603


In [247]:
top_1000_wallets

,wallet,n_deposits,total_deposits_usd,n_withdraws,total_withdraws_usd,n_borrows,total_borrows_usd,n_repays,total_repays_usd,negative_balance,repay_to_borrow_ratio,deposit_to_borrow_ratio,liquidation_ratio,activity_count,credit_score
19490,0x754732d5987d1d7418bba1ae0fc394f489ed05fc,0.0,0,0.0,0,18.0,30,1.0,77638,0.0,2.504452e+03,0.0,0.0,19.0,100.000000
15010,0x84eaeed5c646fe8c65a7d0f913765c9d1d1c5495,0.0,0,17.0,-9223372036854775808,0.0,0,0.0,0,0.0,0.000000e+00,0.0,0.0,17.0,98.689511
12716,0xff33e152a55d9f2364d3b193b5015551822a87c3,2.0,50001000,12.0,-9223372036854775808,0.0,0,0.0,0,0.0,0.000000e+00,50001000.0,0.0,14.0,96.723779
18906,0x57ca561798413a20508b6bc997481e784f3e6e5f,0.0,0,0.0,0,12.0,-9223372036854775808,1.0,1568984,0.0,-1.701096e-13,-0.0,-0.0,13.0,96.068534
21151,0xc629c432925569803f88c58c073d891878edb2a4,0.0,0,0.0,0,8.0,-9223372036854775808,5.0,5000,0.0,-5.421011e-16,-0.0,-0.0,13.0,96.068534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19610,0x7b9bed65c5dd378b96b5c4d94b5b8c862852b514,0.0,0,0.0,0,1.0,4320,1.0,4322,0.0,1.000231e+00,0.0,0.0,2.0,88.860848
2537,0x322e41bf26ccdef8b4b82554e6253d42735d62af,1.0,4966,1.0,4959,0.0,0,0.0,0,0.0,0.000000e+00,4966.0,0.0,2.0,88.860848
21156,0xc6701a83ae9f681994ce5e8ab09073e9e24744e6,0.0,0,0.0,0,1.0,5879,1.0,5880,0.0,1.000000e+00,0.0,0.0,2.0,88.860848
21155,0xc66a6d9b377db1e237455b6c83bee4e64770f911,0.0,0,0.0,0,1.0,5160,1.0,5169,0.0,1.001550e+00,0.0,0.0,2.0,88.860848
